In [88]:
import pandas as pd
import numpy as np
import os

#libreria para el modelo
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

## Carga de Dataframe

Se utiliza un dataframe reducido, ya que en render sólo permite una capacidad limitada en su cuenta free.<br>
Se paso de de un dataframe de 22496 filas a 2250 filas

In [89]:
df1_redu = pd.read_csv(r'dataframe\df1_redu.csv')

In [87]:
df1_redu.shape

(2250, 7)

## Reducción de dataframe

Código para reducir el dataframe, reiniciar los indices y que contenga el **'id'==670290**,<br> para hacer una comparación mas adelante.

In [81]:
#  df1_redu = df1.sample(frac=0.1)
#  df1_redu.reset_index(drop=True, inplace=True)
#  df1_redu[df1_redu['id']==670290]

,genres,title,tags,id,developer,Añio,price
1545,"['Casual', 'Free to Play', 'Indie', 'Simulatio...",Real Pool 3D - Poolians,"['Free to Play', 'Simulation', 'Sports', 'Casu...",670290,Poolians.com,2017,0.0


Guardadar el dataframe reducido

In [83]:
#df1_redu.to_csv(os.path.join('dataframe','df1_redu.csv'), index=False)

## Crea la matriz de Cosine Similarity

In [91]:
# Crear una instancia de TfidfVectorizer con la configuración de palabras vacías en inglés, como 'the', 'a' 
tfidf = TfidfVectorizer (stop_words="english")

# Aplicar la transformación TF-IDF a la columna 'genres' del DataFrame df1_redu
tfidf_matrix = tfidf.fit_transform(df1_redu['genres'])

#Salida de la forma de tfidf_matrix
#Matriz de 22496 filas por 31 columnas - Matriz con el dataframe completo

#Dado que hemos utilizado el vectorizador TF-IDF, el cálculo del producto escalar nos dará directamente 
#la puntuación de similitud del coseno. Por lo tanto, usaremos linear_kernel () de sklearn en lugar de 
#cosine_similarities() ya que es más rápido.

# Calcular la similitud del coseno entre los vectores TF-IDF utilizando linear_kernel
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

# Crear una Serie de pandas que mapea los índices actuales a los 'id' correspondientes
indices = pd.Series (df1_redu.index, index=df1_redu['id']).drop_duplicates()

## Para Guardar la matriz cosine_sim
 Pero pesa demasiado, mas de 500 MB para el dataframe completo

In [ ]:
#Se guarda cosine_sim
#(pd.DataFrame(cosine_sim)).to_parquet(os.path.join('dataframe','cosine_sim.parquet'))
#Para abrir el archivo
#cosine_sim = np.array(pd.read_parquet(r'dataframe\cosine_sim.parquet'))


Del siguiente link se puede descargar un archivo .parquet con la matriz cosine_sim, con el entrenamieto con todos los datos.<br>
link: https://drive.google.com/file/d/18UzYTkwHRVFnN0S3mYILUKBE-BTQmS2j/view?usp=sharing

## Funcion de Sistema de Recomendación

In [92]:
def recomendacion_juego (juego_id, cosine_sim=cosine_sim):

#Obtiene el índice correspondiente al juego_id
    idx = indices[juego_id]

#Obtiene las puntuaciones de similitud del coseno para el juego en cuestión
    sim_scores = list(enumerate (cosine_sim[idx]))

#Ordena las puntuaciones de similitud en orden descendente
    sim_scores = sorted (sim_scores, key=lambda x: x[1], reverse=True)

#Selecciona las 10 puntuaciones de similitud más altas
    sim_scores = sim_scores [1:11]

#Obtiene los índices de las películas recomendadas
    game_indices = [i[0] for i in sim_scores]

#Obtiene la lista de identificadores de juegos recomendados 
    lista = list(df1_redu['id'].iloc[game_indices])
    lista = lista[0:10]
    

    #Para sacar juegos sin repeticion en los recomendados
    a=None
    lista_juegos=[]
    for i,l in enumerate(lista):
        if not(l in lista_juegos) and (l != id):
           lista_juegos.append(lista[i])
    
    #Devuelve un diccionario con las recomendaciones
    return {
          'Juegos recomendados a partir del juego juego_id' : str(juego_id), 
          'No 1': str(df1_redu[df1_redu['id']==lista_juegos[0]].iloc[0][1]),
          'No 2': str(df1_redu[df1_redu['id']==lista_juegos[1]].iloc[0][1]),
          'No 3': str(df1_redu[df1_redu['id']==lista_juegos[2]].iloc[0][1]),
          'No 4': str(df1_redu[df1_redu['id']==lista_juegos[3]].iloc[0][1]),
          'No 5': str(df1_redu[df1_redu['id']==lista_juegos[4]].iloc[0][1])
          }

In [93]:
recomendacion_juego (670290)



{'Juegos recomendados a partir del juego juego_id': '670290',
 'No 1': 'Zaccaria Pinball - Lucky Fruit Table',
 'No 2': 'Zaccaria Pinball - Moon Flight Table',
 'No 3': 'Zaccaria Pinball - Soccer Kings Table',
 'No 4': 'Pixel Puzzles Ultimate - Puzzle Pack: Desserts',
 'No 5': 'Pixel Puzzles Ultimate - Puzzle Pack: Rio'}

Salida producida entrenando el modelo con todos los datos, usando el archivo sim_cosine.parquet

 ```
{'Juegos recomendados a partir del juego juego_id': '670290',
 'No 1': 'Snooker-online multiplayer snooker game!',
 'No 2': "Malzbie's Pinball Collection",
 'No 3': 'Zaccaria Pinball - Blackbelt Table',
 'No 4': 'Zaccaria Pinball - Bronze Membership',
 'No 5': 'Zaccaria Pinball - Cine Star Table'}
```

Como se puede ver ambos casos producen salidas similares, con 3 juegos en común

## Mejoras del código <br>

Se puede mejorar al usar mas columnas para el entrenamiento, al hacer la tfidf.fit_transform se pueden agregar columnas como: tags, developers, title y no sólo genres. Pero esto generaría mayor requerimiento de cómputo 
